In [1]:
import sys
import os
import surprise
import papermill as pm
import scrapbook as sb
import pandas as pd

from surprise.model_selection import GridSearchCV
from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

System version: 3.9.13 (main, Aug 25 2022, 23:26:10) 
[GCC 11.2.0]
Surprise version: 1.1.3


In [2]:
# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

In [31]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"],
    title_col='Title',
    genres_col='Genres',
)

data.head()

100%|███████████████████████████████████████| 4.81k/4.81k [00:36<00:00, 132KB/s]


,userID,itemID,rating,Title,Genres
0,196,242,3.0,Kolya (1996),Comedy
1,63,242,3.0,Kolya (1996),Comedy
2,226,242,5.0,Kolya (1996),Comedy
3,154,242,3.0,Kolya (1996),Comedy
4,306,242,5.0,Kolya (1996),Comedy


In [42]:
train, test = python_random_split(data, 0.75)

# Split data based on columns
train_title = train[["itemID", "Title", "Genres"]]
test_title = test[["itemID", "Title", "Genres"]]

train = train[["userID", "itemID", "rating"]]
test = test[["userID", "itemID", "rating"]]
train_set = surprise.Dataset.load_from_df(train, reader=surprise.Reader(rating_scale=(1, 5)))
train_set

In [47]:
test.head()

,userID,itemID,rating
75721,498,693,3.0
80184,642,542,5.0
19864,58,135,4.0
76699,495,674,3.0
92991,618,735,3.0


In [48]:
param_grid = {
    "n_factors": [50, 100, 150, 200], 
    "n_epochs": [10, 20, 50], 
    "lr_all": [1, 0.1, 0.01, 0.005, 0.0001], 
    "reg_all": [0.0001, 0.001, 0.02, 0.1,  1],
    "verbose":[3]
}
# param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6], "verbose":[3]}
gs = GridSearchCV(surprise.SVD, param_grid, measures=["rmse", "mae"], refit=True, cv=3)

with Timer() as train_time:
    gs.fit(train_set)

print("Took {} seconds for training.".format(train_time.interval))

# best RMSE score
print('best RMSE score', gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print('best parameters ', gs.best_params["rmse"])

print("Took {} seconds for training.".format(train_time.interval))

# use the best model,  retrain on the whole set train
svd_best= gs.best_estimator["rmse"]

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing e

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing e

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing

Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Proce

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing

...

Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing

Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing

Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing

Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing

Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Took 898.2372721569991 seconds for training.
best RMSE score 0.9342488862005313
best parameters  {'n_factors': 200, 'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0.1, 'verbose': 3}
Took 898.2372721569991 seconds for training.


In [49]:
trainset = train_set.build_full_trainset()
# cross_validate(svd_best, trainset, measures=["RMSE", "MAE"], cv=5, verbose=True)

In [50]:
svd_best.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49


In [51]:
predictions = predict(svd_best, test, usercol='userID', itemcol='itemID')
print(predictions.head())
print(test.head())

   userID  itemID  prediction
0     498     693    3.385813
1     642     542    3.083673
2      58     135    3.904893
3     495     674    3.650135
4     618     735    3.555341
       userID  itemID  rating
75721     498     693     3.0
80184     642     542     5.0
19864      58     135     4.0
76699     495     674     3.0
92991     618     735     3.0


In [57]:
df2 = pd.merge(predictions, test,  how='left', left_on=['userID','itemID'], right_on = ['userID','itemID'])
# print(df2.head())
df3 = pd.merge(df2, test_title, how='left', left_on=['itemID'], right_on = ['itemID'])
print(df3.head())

   userID  itemID  prediction  rating
0     498     693    3.385813     3.0
1     642     542    3.083673     5.0
2      58     135    3.904893     4.0
3     495     674    3.650135     3.0
4     618     735    3.555341     3.0
   userID  itemID  prediction  rating          Title Genres
0     498     693    3.385813     3.0  Casino (1995)  Drama
1     498     693    3.385813     3.0  Casino (1995)  Drama
2     498     693    3.385813     3.0  Casino (1995)  Drama
3     498     693    3.385813     3.0  Casino (1995)  Drama
4     498     693    3.385813     3.0  Casino (1995)  Drama


In [59]:
with Timer() as test_time:
    all_predictions = compute_ranking_predictions(svd_best, train, usercol='userID', itemcol='itemID', remove_seen=True)
    
print("Took {} seconds for prediction.".format(test_time.interval))

print(all_predictions.head())

Took 11.628690977000588 seconds for prediction.
       userID  itemID  prediction
75000     405     105    1.002526
75001     405     288    2.653100
75002     405    1012    2.187284
75003     405     253    1.973476
75004     405     826    1.443743
       userID  itemID  rating
98980     405    1305     1.0
69824     314     105     4.0
9928      397     288     4.0
75599     521    1012     3.0
95621     796     795     3.0


In [97]:
# merge dataframes to view the movie details
df4 = pd.merge(all_predictions, train,  how='left', left_on=['userID','itemID'], right_on = ['userID','itemID'])
print(df4.head())
df5 = pd.merge(df4, train_title, how='left', left_on=['itemID'], right_on = ['itemID'])
print(df5.head())

   userID  itemID  prediction  rating
0     405     105    1.002526     NaN
1     405     288    2.653100     NaN
2     405    1012    2.187284     NaN
3     405     253    1.973476     NaN
4     405     826    1.443743     NaN
   userID  itemID  prediction  rating              Title  Genres
0     405     105    1.002526     NaN  Sgt. Bilko (1996)  Comedy
1     405     105    1.002526     NaN  Sgt. Bilko (1996)  Comedy
2     405     105    1.002526     NaN  Sgt. Bilko (1996)  Comedy
3     405     105    1.002526     NaN  Sgt. Bilko (1996)  Comedy
4     405     105    1.002526     NaN  Sgt. Bilko (1996)  Comedy


In [116]:
# print for a particular userID
topk_ids = all_predictions[all_predictions["userID"]==58].sort_values('prediction', ascending = False)
topk_details = pd.merge(topk_ids, train_title,  how='left', left_on=['userID','itemID'], right_on=['itemID'])

# print(topk_details[topk_details["userID"] == 58])
print(topk_details)

ValueError: len(right_on) must equal len(left_on)

In [117]:
eval_rmse = rmse(test, predictions)
eval_mae = mae(test, predictions)
eval_rsquared = rsquared(test, predictions)
eval_exp_var = exp_var(test, predictions)

k = 10
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=k)


print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')

print('----')

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

RMSE:		0.915268
MAE:		0.722139
rsquared:	0.344017
exp var:	0.344242
----
MAP:	0.010064
NDCG:	0.077739
Precision@K:	0.077200
Recall@K:	0.027320
